In [1]:
# Módulos necesarios
import os  # Trabajar con rutas del sistema
import pandas as pd  # Trabajar con DataFrames
import datetime  # Manejo de fechas
from pathlib import Path  # Manejo de rutas
import smtplib  # Envío de correos
from email.mime.text import MIMEText  # Crear correos con texto
from email.mime.multipart import MIMEMultipart  # Correos con múltiples partes
from PyPDF2 import PdfReader  # Leer archivos PDF
import re  # Para validación de correos electrónicos

# 1. Rutas 

In [2]:
Asunto = "NOTIFICACION MOVILIDAD AL REGIMEN SUBSIDIADO EN Septiembre 2025"
Archivo_Name = "Movilidad064_Septiembre_2025"
Fecha = "VIERNES, 10 de Octubre de 2025"
Fecha_Documento = "Yopal, 10 de octubre 2025"

In [3]:
# Rutas
R_Automatico_S1 = r"\\Servernas\AYC2\(01. ASEGURAMIENTO)\01. ASEGURAMIENTO\01. REGIMEN SUBSIDIADO\MUNICIPIOS 2025\MOVILIDAD_064\09_SEPTIEMBRE\MOVILIDAD SEPTIEMBRE"
R_Municipios = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Proyecto INOVA\Colab_Notebooks\dashboard\Base de datos\Codigo DANE\Departamentos.txt"
#R_MS_SIE = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\informes\2025\CTO135.2025 Informe  #3\ACTIVIDAD 19\19.1. Bases de datos notificaciones telefonicas\Validado SIE.xlsx"
R_MS_SIE = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_10_26.csv"
R_Sisben = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SISBEN\2025\08_Agosto\DNP-SISBEN-CO-0000080193.xlsx"
Logo_Capresoca = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\capre.png"
R_Red_Servicios = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\SIE\Aseguramiento\Red De servicios Asignada\Reporte_Red Asignada General_2025_10_26.csv"

# Ruta de salida
output_path = r"\\Servernas\AYC2\(01. ASEGURAMIENTO)\01. ASEGURAMIENTO\01. REGIMEN SUBSIDIADO\MUNICIPIOS 2025\MOVILIDAD_064\09_SEPTIEMBRE\CORREOS USUARIOS\User_Df.TXT"
SAVE_DIR = r"\\Servernas\AYC2\(01. ASEGURAMIENTO)\01. ASEGURAMIENTO\01. REGIMEN SUBSIDIADO\MUNICIPIOS 2025\MOVILIDAD_064\09_SEPTIEMBRE\CORREOS USUARIOS"

In [4]:
Contracena = "None"
Correo = "aseg13@capresoc.onmicrosoft.com"

# 2. Lectura de Datafarmes

In [5]:
# Obtener todos los archivos .VAL en la ruta
val_files = [f for f in os.listdir(R_Automatico_S1) if f.endswith('.VAL')]

# Leer los archivos .VAL en un DataFrame
dataframes = []
for file in val_files:
    file_path = os.path.join(R_Automatico_S1, file)
    df = pd.read_csv(file_path, sep=',', encoding='ansi', header=None, dtype=str)
    dataframes.append(df)

# Concatenar todos los DataFrames en uno solo
df_Auto_S1 = pd.concat(dataframes, ignore_index=True)
# Cargar el archivo .txt en un DataFrame
df_municipios = pd.read_csv(R_Municipios, sep=';', encoding='utf-8', header=0, dtype=str)
df_Red_Servicios = pd.read_csv(R_Red_Servicios, sep=';', encoding='ANSI', header=0, dtype=str)

# Cargar el archivo Excel en un DataFrame, leyendo todos los datos como tipo str
#df_ms_sie = pd.read_excel(R_MS_SIE, sheet_name="Sheet1", dtype=str)
df_ms_sie = pd.read_csv(R_MS_SIE, sep=';', encoding='ANSI', header=0, dtype=str)

# Cargar el archivo Excel en un DataFrame, leyendo todos los datos como tipo str
df_Sisben= pd.read_excel(R_Sisben, sheet_name="Resultado", dtype=str)

In [6]:
print(df_Red_Servicios.columns.tolist())
df_Red_Servicios = df_Red_Servicios.drop(columns=["nit", "codigo", "Unnamed: 7"])
print("Nuevas columnas en df_Red_Servicios:", df_Red_Servicios.columns.tolist())

['nit', 'razon_social', 'abreviatura', 'numero_identificacion', 'municipio', 'codigo', 'servicio', 'Unnamed: 7']
Nuevas columnas en df_Red_Servicios: ['razon_social', 'abreviatura', 'numero_identificacion', 'municipio', 'servicio']


In [7]:
print(df_Red_Servicios["servicio"].unique())
print("Cantidad de registros antes:", df_Red_Servicios.shape[0])

['MEDICAMENTOS' 'OPTOMETRIA' 'ODONTOLOGIA GENERAL' 'LABORATORIO CLINICO'
 'MEDICINA GENERAL' 'MEDICAMENTOS ESPECIALIZADOS' 'OFTALMOLOGIA'
 'ANESTESIA' 'SERVICIO FARMACEUTICO' 'ENFERMERIA'
 'FONOAUDIOLOGIA Y/O TERAPIA DEL LENGUAJE' 'TERAPIA OCUPACIONAL'
 'TERAPIA RESPIRATORIA' 'FISIOTERAPIA' 'MEDICINA FAMILIAR'
 'PROMOCION Y PREVENCION GENERAL' 'RUTA MATERNO PERINATAL'
 'SERVICIO DE DEMANDA INDUCIDA A LOS PROGRAMAS DE DETECCION TEMPRANA Y PROTECCION ESPECIFICA'
 'PROCEDIMIENTOS MENORES' 'HOSPITALIZACION GENERAL'
 'TRANSPORTE ASISTENCIAL MEDICALIZADO' 'SERVICIO DE URGENCIAS'
 'TRANSPORTE ASISTENCIAL BASICO' 'RAYOS X'
 'RUTA PROMOCION Y MANTENIMIENTO' 'IMAGENES DIAGNOSTICAS - IONIZANTES'
 'PROTECCION ESPECIFICA - VACUNACION' 'NUTRICION Y DIETETICA'
 'TOMA DE MUESTRAS DE LABORATORIO CLINICO' 'CIRUGIA ORTOPEDICA'
 'INFECTOLOGIA']
Cantidad de registros antes: 2937508


In [8]:
df_Red_Servicios = df_Red_Servicios[df_Red_Servicios['servicio'].isin(['MEDICINA GENERAL', 'ODONTOLOGIA GENERAL'])]
print("Cantidad de registros antes:", df_Red_Servicios.shape[0])

Cantidad de registros antes: 382013


In [9]:
from tabulate import tabulate

registro = df_Red_Servicios[df_Red_Servicios['numero_identificacion'] == "1246129076"]
# Filtrar el registro donde 'numero_identificacion' es "1116542442"
print(tabulate(registro, headers='keys', tablefmt='psql'))


+---------+---------------------------------------+---------------+-------------------------+-------------+---------------------+
|         | razon_social                          | abreviatura   |   numero_identificacion | municipio   | servicio            |
|---------+---------------------------------------+---------------+-------------------------+-------------+---------------------|
| 1285838 | EMPRESA SOCIAL DEL ESTADO SALUD YOPAL | RC            |              1246129076 | Yopal       | ODONTOLOGIA GENERAL |
| 1285842 | EMPRESA SOCIAL DEL ESTADO SALUD YOPAL | RC            |              1246129076 | Yopal       | MEDICINA GENERAL    |
+---------+---------------------------------------+---------------+-------------------------+-------------+---------------------+


In [10]:
print("Cantidad de registros antes:", df_Red_Servicios.shape[0])

df_Red_Servicios = df_Red_Servicios.pivot_table(
    index=['abreviatura', 'numero_identificacion'],
    columns='servicio',
    values='razon_social',
    aggfunc='first'
).reset_index()

print("Cantidad de registros después:", df_Red_Servicios.shape[0])
df_Red_Servicios

Cantidad de registros antes: 382013
Cantidad de registros después: 197661


servicio,abreviatura,numero_identificacion,MEDICINA GENERAL,ODONTOLOGIA GENERAL
0,AS,85001A0001,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL
1,AS,85001D001,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,NaN
2,AS,85001D006,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,NaN
3,AS,85001D007,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,NaN
4,AS,85001D037,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,NaN
...,...,...,...,...
197656,TI,RAN1117323746,RED SALUD CASANARE E.S.E,RED SALUD CASANARE E.S.E
197657,TI,RAN1118776184,RED SALUD CASANARE E.S.E,RED SALUD CASANARE E.S.E
197658,TI,RAN1120361085,RED SALUD CASANARE E.S.E,RED SALUD CASANARE E.S.E
197659,TI,W1W0250799,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,NaN


In [11]:
# Crear la columna "Código Municipio" en df_Auto_S1 unificando las columnas 17 y 18
df_Auto_S1['Código Municipio'] = df_Auto_S1[17].str.zfill(2) + df_Auto_S1[18].str.zfill(3)

# Realizar el merge para traer la columna "Nombre Municipio" desde df_municipios
df_Auto_S1 = df_Auto_S1.merge(df_municipios[['CODIGO', 'Nombre Municipio']],
                              left_on='Código Municipio', right_on='CODIGO', how='left')

# Eliminar la columna 'CODIGO' que se agregó durante el merge
df_Auto_S1 = df_Auto_S1.drop(columns=['CODIGO'])

In [12]:
print(f"La cantidad de registros en df_Auto_S1 es: {df_Auto_S1.shape[0]}")
df_Auto_S1 = df_Auto_S1.drop_duplicates(subset=[1, 2])
df_Auto_S1_filtered = df_Auto_S1[df_Auto_S1[32] == "EPS025"]
df_Auto_S1_filtered = df_Auto_S1[df_Auto_S1[17] == 85]
print(f"La cantidad de registros en df_Auto_S1 es: {df_Auto_S1.shape[0]}")

La cantidad de registros en df_Auto_S1 es: 2840
La cantidad de registros en df_Auto_S1 es: 2828


In [13]:
# Unir df_Auto_S1 con df_ms_sie para traer la columna correo_electronico
df_Auto_S1 = df_Auto_S1.merge(df_ms_sie[['numero_identificacion', 'correo_electronico']],
                              left_on=2, right_on='numero_identificacion', how='left')

# Eliminar la columna 'numero_identificacion' que se agregó durante el merge
df_Auto_S1 = df_Auto_S1.drop(columns=['numero_identificacion'])

In [14]:
num_empty = df_Auto_S1['correo_electronico'].isna().sum()
num_not_empty = df_Auto_S1['correo_electronico'].notna().sum()

print(f"Número de registros vacíos en 'correo_electronico': {num_empty}")
print(f"Número de registros no vacíos en 'correo_electronico': {num_not_empty}")

Número de registros vacíos en 'correo_electronico': 538
Número de registros no vacíos en 'correo_electronico': 2292


In [15]:
# Mostrar la cantidad de registros vacíos antes del proceso
print(f"Registros vacíos en 'correo_electronico' antes del proceso: {df_Auto_S1['correo_electronico'].isna().sum()}")

# Filtrar los valores no deseados en la columna 'correo_electronico'
df_Auto_S1 = df_Auto_S1[~df_Auto_S1['correo_electronico'].str.contains(r'actua|notie|sincorr', case=False, na=False)]
df_Auto_S1['correo_electronico'] = df_Auto_S1['correo_electronico'].apply(
    lambda email: "" if isinstance(email, str) and (
        re.search(r'sincorreo', email, re.IGNORECASE) or 
        re.search(r'notiene', email, re.IGNORECASE) or 
        re.search(r'TRASLADO', email, re.IGNORECASE) or 
        re.search(r'ACTUALIZAR', email, re.IGNORECASE) or 
        re.search(r'actualiza', email, re.IGNORECASE) or 
        re.search(r'actualizar', email, re.IGNORECASE) or 
        re.search(r'tiene', email, re.IGNORECASE) or 
        re.search(r'TIENE', email, re.IGNORECASE) or 
        re.search(r'afilia', email, re.IGNORECASE) or 
        re.search(r'traslado', email, re.IGNORECASE) or 
        email.split('@')[0].isdigit()
    ) else email
)

# Mostrar la cantidad de registros vacíos después del proceso
print(f"Registros vacíos en 'correo_electronico' después del proceso: {df_Auto_S1['correo_electronico'].isna().sum()}")

Registros vacíos en 'correo_electronico' antes del proceso: 538
Registros vacíos en 'correo_electronico' después del proceso: 538


In [16]:
# Mostrar la cantidad de registros vacíos antes del proceso
print(f"Registros vacíos en 'correo_electronico' antes del proceso: {df_Auto_S1['correo_electronico'].isna().sum()}")

# Realizar el merge para buscar correos en df_Sisben
df_Auto_S1 = df_Auto_S1.merge(df_Sisben[['numeroDocumento', 'email_contacto']],
                              left_on=2, right_on='numeroDocumento', how='left')

# Rellenar los valores vacíos en 'correo_electronico' con los valores de 'email_contacto'
df_Auto_S1['correo_electronico'] = df_Auto_S1['correo_electronico'].fillna(df_Auto_S1['email_contacto'])

# Eliminar la columna 'email_contacto' y 'numeroDocumento' que se agregó durante el merge
df_Auto_S1 = df_Auto_S1.drop(columns=['email_contacto', 'numeroDocumento'])

# Mostrar la cantidad de registros vacíos después del proceso
print(f"Registros vacíos en 'correo_electronico' después del proceso: {df_Auto_S1['correo_electronico'].isna().sum()}")

Registros vacíos en 'correo_electronico' antes del proceso: 538
Registros vacíos en 'correo_electronico' después del proceso: 351


In [17]:
# Mostrar la cantidad de registros vacíos antes del proceso
print(f"Registros vacíos en 'correo_electronico' antes del proceso: {df_Auto_S1['correo_electronico'].isna().sum()}")

# Filtrar los valores no deseados en la columna 'correo_electronico'
df_Auto_S1 = df_Auto_S1[~df_Auto_S1['correo_electronico'].str.contains(r'actua|notie|sincorr', case=False, na=False)]
df_Auto_S1['correo_electronico'] = df_Auto_S1['correo_electronico'].apply(
    lambda email: "" if isinstance(email, str) and (
        re.search(r'sincorreo', email, re.IGNORECASE) or 
        re.search(r'notiene', email, re.IGNORECASE) or 
        re.search(r'TRASLADO', email, re.IGNORECASE) or 
        re.search(r'ACTUALIZAR', email, re.IGNORECASE) or 
        re.search(r'actualiza', email, re.IGNORECASE) or 
        re.search(r'actualizar', email, re.IGNORECASE) or 
        re.search(r'tiene', email, re.IGNORECASE) or 
        re.search(r'TIENE', email, re.IGNORECASE) or 
        re.search(r'afilia', email, re.IGNORECASE) or 
        re.search(r'traslado', email, re.IGNORECASE) or 
        email.split('@')[0].isdigit()
    ) else email
)

# Mostrar la cantidad de registros vacíos después del proceso
print(f"Registros vacíos en 'correo_electronico' después del proceso: {df_Auto_S1['correo_electronico'].isna().sum()}")

Registros vacíos en 'correo_electronico' antes del proceso: 351
Registros vacíos en 'correo_electronico' después del proceso: 351


In [18]:
df_Auto_S1 = df_Auto_S1.dropna(subset=['correo_electronico'])

In [19]:
# Función para corregir un correo electrónico
def fix_email(email):
    if pd.isna(email):
        return email
    # Quitar todos los espacios
    new_email = email.replace(" ", "")
    # Si el correo termina en "com." o "COM." (ignorando mayúsculas) quitar el punto final
    if new_email.lower().endswith("com."):
        new_email = new_email[:-1]
    return new_email

# Aplicar la corrección y contar cuántos registros se modificaron
original_emails = df_Auto_S1['correo_electronico']
corrected_emails = original_emails.apply(fix_email)
num_corrected = (original_emails != corrected_emails).sum()

# Actualizar la columna con las correcciones
df_Auto_S1['correo_electronico'] = corrected_emails

print(f"Se corrigieron {num_corrected} registros en 'correo_electronico'.")

Se corrigieron 4 registros en 'correo_electronico'.


In [20]:
df_Auto_S1 = df_Auto_S1.merge(
    df_Red_Servicios[['numero_identificacion', 'MEDICINA GENERAL', 'ODONTOLOGIA GENERAL']],
    left_on=10,
    right_on='numero_identificacion',
    how='left'
)
# (Optional) Drop the duplicate 'numero_identificacion' column if no longer needed:
df_Auto_S1 = df_Auto_S1.drop(columns=['numero_identificacion'])

In [21]:
mask = (
    df_Auto_S1['MEDICINA GENERAL'].notna() &
    (df_Auto_S1['MEDICINA GENERAL'].astype(str).str.strip() != '') &
    df_Auto_S1['ODONTOLOGIA GENERAL'].notna() &
    (df_Auto_S1['ODONTOLOGIA GENERAL'].astype(str).str.strip() != '')
)
df_Auto_S1 = df_Auto_S1[mask]

print("Cantidad de registros después del filtrado:", df_Auto_S1.shape[0])

Cantidad de registros después del filtrado: 1697


In [22]:
df_Auto_S1

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,Código Municipio,Nombre Municipio,correo_electronico,MEDICINA GENERAL,ODONTOLOGIA GENERAL
0,EPS025,CC,1000579442,LUCUMI,CORTES,LIBNI,VIVIANA,12/11/2001,F,CC,...,COL,85001,COL,F,NaN,85001,YOPAL,no@gmail.com,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL
1,EPS025,CC,1002270577,FUENTES,ALVARADO,LINA,MARCELA,08/11/2001,F,CC,...,COL,85125,COL,F,NaN,85125,HATO COROZAL,linamarcela08910@gmail.com,RED SALUD CASANARE E.S.E,RED SALUD CASANARE E.S.E
2,EPS025,CC,1002603243,PAEZ,PATIÑO,YAKELINE,NaN,27/10/1994,F,CC,...,COL,85015,COL,F,NaN,85015,CHÁMEZA,yakes-97@hotmail.com,RED SALUD CASANARE E.S.E,RED SALUD CASANARE E.S.E
3,EPS025,CC,1002645522,PACAGUI,AFRICANO,YOHN,ALEXANDER,26/12/2001,M,CC,...,COL,85001,COL,M,NaN,85001,YOPAL,pacaguiafricano@gmail.com,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL
4,EPS025,CC,1002758775,ZARATE,SARMIENTO,ABNER,ELIUD,05/02/2003,M,CC,...,COL,85410,COL,M,NaN,85410,TAURAMENA,ELIUDSARMIENTO19@GMAIL.COM,E.S.E HOSPITAL LOCAL DE TAURAMENA,E.S.E HOSPITAL LOCAL DE TAURAMENA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735,EPS025,CC,74301649,RINCON,GONZALEZ,OMAR,ALBERTO,18/09/1971,M,CC,...,COL,85010,COL,M,NaN,85010,AGUAZUL,manoklito@yahoo.es,HOSPITAL DE AGUAZUL JUAN HERNANDO URREGO E.S.E,HOSPITAL DE AGUAZUL JUAN HERNANDO URREGO E.S.E
1736,EPS025,CC,1115911152,AREVALO,CUBIDES,MAILY,JERAXY,13/03/2006,F,CC,...,COL,85010,COL,F,NaN,85010,AGUAZUL,mailyjeraxy13@gmail.com,HOSPITAL DE AGUAZUL JUAN HERNANDO URREGO E.S.E,HOSPITAL DE AGUAZUL JUAN HERNANDO URREGO E.S.E
1737,EPS025,CC,79271536,ALBA,GOMEZ,ORLANDO,NaN,28/11/1962,M,CC,...,COL,11001,COL,M,NaN,85250,PAZ DE ARIPORO,orlandogomez536@gmail.com,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL
1738,EPS025,RC,1222147094,ARAQUE,VARGAS,EMILIANO,NaN,06/03/2025,M,RC,...,COL,85001,COL,M,NaN,85001,YOPAL,anavargas0796@gmail.com,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL,EMPRESA SOCIAL DEL ESTADO SALUD YOPAL


In [23]:
df_Auto_S1 = df_Auto_S1.drop_duplicates(subset=[9, 10])
print("Cantidad de registros únicos según el id (columnas 9 y 10):", df_Auto_S1.shape[0])

Cantidad de registros únicos según el id (columnas 9 y 10): 1609


In [24]:
# Guardar el DataFrame df_Auto_S1 en la ruta especificada
df_Auto_S1.to_csv(output_path, sep=',', encoding='ansi', index=False)

# 2 prueba 

In [25]:
import os
import ctypes

# 1) Asegurarte de apuntar a la carpeta de MSYS2 mingw64\bin
os.add_dll_directory(r"C:\msys64\mingw64\bin")

# 2) (Opcional) Cargar manualmente una o varias DLL si deseas comprobar:
dll_gobject = ctypes.CDLL("libgobject-2.0-0.dll")
dll_pangoft2 = ctypes.CDLL("libpangoft2-1.0-0.dll")
dll_pango   = ctypes.CDLL("libpango-1.0-0.dll")
dll_fontconfig = ctypes.CDLL("libfontconfig-1.dll")
# ... etc.

# 3) Recién ahora importar las librerías que usarán WeasyPrint o GTK
import pandas as pd
import smtplib
import datetime
import pdfkit
from weasyprint import HTML
from email import generator
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from fpdf import FPDF



# Si deseas usar win32com (automatización de Outlook) en lugar de smtplib, cambia a True.
USE_WIN32COM = True

if USE_WIN32COM:
    import win32com.client as win32

# ------------------------------------------------------------------------
# CONFIGURACIÓN DE CORREO (SMTP / Outlook)
# ------------------------------------------------------------------------
# Para smtplib (si USE_WIN32COM es False):
SMTP_SERVER = "smtp-mail.outlook.com"  # Ejemplo para Outlook
SMTP_PORT = 587
#SMTP_USERNAME = "rincon3259@gmail.com"
#SMTP_PASSWORD = "hdyx atvw ncdq otgw"  # Contraseña de aplicación si tienes 2FA "rincon3259@gmail.com"
SMTP_USERNAME = Correo
SMTP_PASSWORD = Contracena
FROM_NAME = "Capresoca EPS"  # Cómo se mostrará el remitente

# Correo principal y copia
# Ahora, cada correo se enviará a la dirección especificada en la columna "correo_electronico"
# de cada fila del DataFrame Df_Prueba, por lo que no es necesario definir un TO_EMAIL fijo.
#TO_EMAIL = ""
CC_EMAIL = ""

#TO_EMAIL = "rincon3259@gmail.com"
#CC_EMAIL = "marcopolo3259@gmail.com"

# ------------------------------------------------------------------------
# ENCABEZADO (HTML) + CUERPO DEL MENSAJE
# ------------------------------------------------------------------------
EMAIL_TEMPLATE = """\
<html>
<head>
    <meta charset="utf-8">
</head>
<body style="font-family: Arial, sans-serif; font-size: 14px;">

    <!-- Encabezado con imagen a la izquierda y textos a la derecha -->
    <table width="100%" style="border-collapse: collapse;">
    <tr>
        <!-- Columna izquierda: imagen + NIT + EPS -->
        <td width="150" style="vertical-align: top;">
            <img src="cid:capre.png" alt="Capresoca"
                width="150"
                style="width:150px; height:auto; display:block;"
                border="0">
            <br><br>
            <span style="font-family: Arial, sans-serif; font-size: 14px;">
                NIT. 891.856.000-7<br>
                EPS en intervención
            </span>
        </td>
        <!-- Columna derecha: textos centrados -->
        <td style="vertical-align: top; text-align: center;">
        <span style="font-family: Arial, sans-serif; font-size: 16px; font-weight: bold;">
            COMUNICACIÓN INSTITUCIONAL
        </span><br><br>
        <span style="font-family: Arial, sans-serif; font-size: 14px;">FO-GD-01</span><br>
        <span style="font-family: Arial, sans-serif; font-size: 14px;">2024-01-19</span><br>
        <span style="font-family: Arial, sans-serif; font-size: 14px;">V.03</span>
        </td>
    </tr>
    </table>

    <hr>

    <!-- Cuerpo del mensaje -->
    <p>{Fecha_Documento}</p>
    <p>
        Señor (a)<br>
        <strong>{col5} {col6} {col3} {col4}</strong><br>
        <strong>{col1} {col2}</strong><br>
        <strong>{municipio} {cod_municipio}</strong>
    </p>
    <p><strong>Ref.: {Asunto}</strong></p>
    <p>
        Cordial saludo,<br><br>
        Comprometidos con el Sistema General de Seguridad Social en Salud (SGSSS), atendiendo, entre otros,
        los principios de universalidad, continuidad y, considerando que usted no reporta la novedad de Movilidad a pesar
        de cumplir con los requisitos para ello (Tener Sisbén metodología IV actualizada, o pertenecer a listado censal),
        se está dando aplicación a lo definido en la normativa vigente para mitigar el riesgo y/o afectación en la
        continuidad de la afiliación, como de la prestación de los servicios de salud.
    </p>
    <p>
        Por lo anterior y en virtud del debido proceso, formalmente le comunicamos que, a la fecha conforme a lo
        definido por el Ministerio de Salud y Protección Social en Salud, en el Decreto 064 de 2020:<br>
        Artículo 7º- modificación del artículo 2.1.7.8 del Decreto 780 de 2016 ‘registro y reporte de la novedad de
        movilidad’:
        <em>“(…) Cuando el usuario no registre la solicitud de movilidad del régimen contributivo al régimen subsidiado,
        la EPS deberá reportarla en la BDUA e informar al afiliado y a la respectiva entidad territorial, tal novedad (…)”</em>.
    </p>
    <p>
        En efecto, Capresoca EPS, le informa que se aplicó la novedad de Movilidad al Régimen Subsidiado y continúan con
        la prestación de los servicios de salud en nuestra EPS y la IPS que le prestará los servicios de salud en
        medicina general es {Medicina} y los servicios de odontología es {Odontologia}. 
        Para más información puede ser consultada en los móviles: 3106805416 – 3229467854.
    </p>
    <p>
        Asimismo, los afiliados tienen la posibilidad de consultar su red de servicios de salud a través del siguiente
        enlace:
        <a href="http://capresoca.gov.co:/app_sie/loginAffiliate.xhtml#nbb">
        http://capresoca.gov.co:/app_sie/loginAffiliate.xhtml#nbb</a>.
        Para acceder, deben seleccionar la opción “CREACIÓN de USUARIO AFILIADO” e ingresar los datos básicos necesarios
        para obtener una clave que les permitirá consultar y descargar información sobre los servicios disponibles
        en la red primaria de salud.
    </p>
    <p>
        <img src="cid:yesid_firma.png" alt="Firma" style="height: 100px;"><br><br>
        <strong>Osmar Yesid Rincon Zorro, Profesional de Apoyo de Aseguramiento</strong>
    </p>
</body>
</html>
"""

# ------------------------------------------------------------------------
# FUNCIÓN PARA ENVIAR EL CORREO
# ------------------------------------------------------------------------
def send_email_with_header(row):
    """
    Envía un correo para la fila `row` del DataFrame utilizando la plantilla HTML.
    Si USE_WIN32COM es True se utiliza Outlook mediante win32com, de lo contrario se usa smtplib.
    
    Por ahora se envía a un correo fijo (TO_EMAIL) con copia (CC_EMAIL). 
    La lógica para usar row["correo_electronico"] está comentada para este ejemplo.
    """
    to_email = row["correo_electronico"]
    cc_email = CC_EMAIL

    # Usar la variable global Asunto en el formateo de la plantilla.
    email_body = EMAIL_TEMPLATE.format(
        col5=row[5],
        col6=row[6],
        col3=row[3],
        col4=row[4],
        col1=row[1],
        col2=row[2],
        municipio=row["Nombre Municipio"],
        cod_municipio=row["Código Municipio"],
        Medicina=row["MEDICINA GENERAL"],
        Odontologia=row["ODONTOLOGIA GENERAL"],
        Asunto=Asunto,
        Fecha_Documento = Fecha_Documento
    )

    if USE_WIN32COM:
        try:
            outlook = win32.Dispatch('Outlook.Application')
            mail = outlook.CreateItem(0)  # 0: MailItem
            mail.To = to_email
            mail.CC = cc_email
            mail.Subject = "Notificación Movilidad al Régimen Subsidiado"
            mail.HTMLBody = email_body
            
            try:
                with open(r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\capre.png", 'rb') as f:
                    img_data = f.read()
                image = mail.Attachments.Add(r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\capre.png")
                image.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", "capre.png")
            except Exception as e_img:
                print(f"Error al incrustar capre.png: {e_img}")
            
            try:
                with open(r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\YESID.png", 'rb') as f:
                    firma_data = f.read()
                firma_image = mail.Attachments.Add(r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\YESID.png")
                firma_image.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", "yesid_firma.png")
            except Exception as e_firma:
                print(f"Error al incrustar YESID.png: {e_firma}")

            mail.Send()
            print(f"Correo enviado con éxito a {to_email} (CC: {cc_email}) vía Outlook.")
        except Exception as e:
            print(f"Error al enviar correo vía Outlook: {e}")
    else:
        msg = MIMEMultipart('related')
        msg['Subject'] = "Notificación Movilidad al Régimen Subsidiado"
        msg['From'] = f"{FROM_NAME} <{SMTP_USERNAME}>"
        msg['To'] = to_email
        msg['Cc'] = cc_email
        to_addresses = [to_email, cc_email]

        msg.attach(MIMEText(email_body, 'html'))

        try:
            with open(r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\capre.png", 'rb') as f:
                img_data = f.read()
            image = MIMEImage(img_data, name='capre.png')
            image.add_header('Content-ID', '<capre.png>')
            msg.attach(image)
        except Exception as e_img:
            print(f"Error al leer la imagen capre.png: {e_img}")

        try:
            with open(r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\YESID.png", 'rb') as f:
                firma_data = f.read()
            firma_image = MIMEImage(firma_data, name='YESID.png')
            firma_image.add_header('Content-ID', '<yesid_firma.png>')
            msg.attach(firma_image)
        except Exception as e_firma:
            print(f"Error al leer la imagen YESID.png: {e_firma}")

        try:
            with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
                server.starttls()
                server.login(SMTP_USERNAME, SMTP_PASSWORD)
                server.send_message(msg, from_addr=msg['From'], to_addrs=to_addresses)
            print(f"Correo enviado con éxito a {to_email} (CC: {cc_email}) vía smtplib.")
        except Exception as e:
            print(f"Error al enviar el correo a {to_email}: {e}")

# ------------------------------------------------------------------------
# FUNCIÓN PARA GENERAR PDF POR MUNICIPIO
# ------------------------------------------------------------------------
def generate_pdf_for_municipality(municipality_rows, municipality_name):
    """
    Crea un PDF que contiene el contenido HTML completo (con imágenes) de todos los correos
    para el municipio 'municipality_name'. Cada correo se muestra en una página separada
    e incluye la información de envío (De, Enviado el, Para y Asunto).

    Se guardará en: SAVE_DIR/Correo_{municipality_name}_Movilidad064_Diciembre_2024.pdf

    Se usa WeasyPrint para generar el PDF.
    """
    # Rutas de las imágenes (asegúrate que sean correctas)
    Logo_Capresoca = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\capre.png"
    yesid_firma_path = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Imágenes\YESID.png"
    
    pdf_filename = f"Correo_{municipality_name}_{Archivo_Name}.pdf"
    pdf_path = os.path.join(SAVE_DIR, pdf_filename)
    
    # CSS para estilos y para forzar salto de página en cada correo
    css = """
    <style>
      .correo-container {
        page-break-after: always;
        margin-bottom: 20px;
      }
      .meta-info {
        font-family: Arial, sans-serif;
        font-size: 14px;
        margin-bottom: 10px;
      }
      .meta-info p {
        margin: 2px 0;
      }
    </style>
    """
    
    # Se inicia el HTML con CSS
    html_contents = css

    # Recorremos cada registro para armar su contenido
    for idx, row in municipality_rows.iterrows():
        # Variables de metadatos: Se usa la variable global Fecha y Asunto
        de_val = row.get("De", "ASEGURAMIENTO CAPRESOCA EPS")
        fecha_val = row.get("FechaEnvio", Fecha)  # Utiliza la variable global Fecha
        asunto_val = row.get("Asunto", Asunto)      # Utiliza la variable global Asunto
        para_val = row.get("correo_electronico", "alegunamaria2014@gmail.com")
        
        # Bloque de metadatos del correo
        meta_html = f"""
        <div class="meta-info">
            <p><strong>De:</strong> {de_val}</p>
            <p><strong>Enviado el:</strong> {fecha_val}</p>
            <p><strong>Para:</strong> {para_val}</p>
            <p><strong>Asunto:</strong> {asunto_val}</p>
        </div>
        """
        
        # Construir el cuerpo del correo usando la plantilla existente.
        # Se agrega Asunto=Asunto para formatear la plantilla global, la cual utiliza {Asunto}.
        email_html = EMAIL_TEMPLATE.format(
            col5=row[5],
            col6=row[6],
            col3=row[3],
            col4=row[4],
            col1=row[1],
            col2=row[2],
            municipio=row["Nombre Municipio"],
            cod_municipio=row["Código Municipio"],
            Medicina=row["MEDICINA GENERAL"],
            Odontologia=row["ODONTOLOGIA GENERAL"],
            Asunto=Asunto,
            Fecha_Documento = Fecha_Documento
        )
        # Reemplazar las referencias CID por rutas absolutas usando "file:///"
        email_html = email_html.replace('cid:capre.png', "file:///" + Logo_Capresoca.replace("\\", "/"))
        email_html = email_html.replace('cid:yesid_firma.png', "file:///" + yesid_firma_path.replace("\\", "/"))
        
        # Envolver el contenido del correo en un contenedor que forza salto de página
        correo_html = f"""
        <div class="correo-container">
            {meta_html}
            {email_html}
        </div>
        """
        
        html_contents += correo_html
    
    try:
        # Generar el PDF usando WeasyPrint
        HTML(string=html_contents, base_url='file:///' + os.path.abspath('.')).write_pdf(pdf_path)
        print(f"PDF generado para municipio {municipality_name}: {pdf_path}")
    except Exception as e:
        print(f"Error al generar el PDF: {e}")


# ------------------------------------------------------------------------
# PROCESO PRINCIPAL
# ------------------------------------------------------------------------
def main():
    
    # 1) Enviar un correo por cada registro
    for index, row in df_Auto_S1.iterrows():
        send_email_with_header(row)

    # 2) Generar un PDF por cada municipio
    grouped = df_Auto_S1.groupby("Nombre Municipio")
    for municipality_name, group_df in grouped:
        generate_pdf_for_municipality(group_df, municipality_name)

    print("Proceso completado.")

if __name__ == "__main__":
    main()

Correo enviado con éxito a no@gmail.com (CC: ) vía Outlook.
Correo enviado con éxito a linamarcela08910@gmail.com (CC: ) vía Outlook.
Correo enviado con éxito a yakes-97@hotmail.com (CC: ) vía Outlook.
Correo enviado con éxito a pacaguiafricano@gmail.com (CC: ) vía Outlook.
Correo enviado con éxito a ELIUDSARMIENTO19@GMAIL.COM (CC: ) vía Outlook.
Correo enviado con éxito a alecondia245@gmail.com (CC: ) vía Outlook.
Correo enviado con éxito a jhonardila@hotmail.com (CC: ) vía Outlook.
Correo enviado con éxito a ad4469237@gmail.com (CC: ) vía Outlook.
Correo enviado con éxito a carlosvides@hotmail.com (CC: ) vía Outlook.
Correo enviado con éxito a JHONFREDY2014-2015@HOTMAIL.COM (CC: ) vía Outlook.
Correo enviado con éxito a sebastian-0550@outlook.com (CC: ) vía Outlook.
Correo enviado con éxito a av3460318@gmail.com (CC: ) vía Outlook.
Correo enviado con éxito a caperatapiaeilsa@gmail.com (CC: ) vía Outlook.
Correo enviado con éxito a jhonrc1507@gmail.com (CC: ) vía Outlook.
Correo envia